In [1]:

%cd ..
root_dir = !pwd


/home/jabcross/repos/iara


/usr/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%cd {root_dir[0]}
!pwd

exec_path = "/home/jabcross/repos/preesm-apps/SIFT/Code/Debug/bin/extract"
kernels = !grep 'iara.actor' 'test/Iara/05-cholesky/topology.mlir' |\
        grep -oE '@[a-zA-Z_]+'
if '@run' in kernels:
  kernels.remove('@run')

kernels = [k[1:] for k in kernels]

print(kernels)



/home/jabcross/repos/iara
/home/jabcross/repos/iara
['kernel_potrf', 'kernel_trsm', 'kernel_gemm', 'kernel_syrk', 'kernel_split', 'kernel_join']


In [41]:
# profile
# Using sudo to be able to get all core information.
# Be careful with the perf.data, it may have personal information.

%cd {root_dir[0]}
%cd profile
%mkdir work
%cd work

dir = !dirname {exec_path}
print('dir: ', dir[0])
%cd {dir[0]}

!pwd
command = f'perf record \
               -e cycles \
               -e sched:sched_switch \
               --switch-events \
               --sample-cpu \
               --call-graph dwarf \
               -a -g -T -F 99 ./extract && \
               chown "$USER:$USER" perf.data && \
               chmod ugo+rw perf.data'
# !run0 sh -x -c '{command}'

!mv perf.data {root_dir[0]}/profile/work

!pwd



/home/jabcross/repos/iara
/home/jabcross/repos/iara/profile
mkdir: cannot create directory ‘work’: File exists
/home/jabcross/repos/iara/profile/work
dir:  /home/jabcross/repos/preesm-apps/SIFT/Code/Debug/bin
/home/jabcross/repos/preesm-apps/SIFT/Code/Debug/bin
/home/jabcross/repos/preesm-apps/SIFT/Code/Debug/bin
/home/jabcross/repos/preesm-apps/SIFT/Code/Debug/bin


In [4]:
%cd {root_dir[0]}/profile/work
!ls

!perf data convert --to-json perf-data.json

import json
import fileinput

with open("perf-data.json") as file:
  data = json.load(file)

print(len(data))


/home/jabcross/repos/iara/profile/work
output.svg  perf.data  perf-data.json


Output file exists. Use --force to overwrite it.


/usr/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


3


In [36]:
import pandas as pd

df = pd.json_normalize(data['samples'])

df[(df['comm']=='extract')&(len(df['callchain']) > 0)]





,timestamp,pid,tid,cpu,comm,callchain,prev_comm,prev_pid,prev_prio,prev_state,next_comm,next_pid,next_prio
158,118567340780477,795984,795986,1,extract,"[{'ip': '0xffffffffa45b0cc3', 'symbol': 'mas_w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,118567341118682,795984,795987,2,extract,"[{'ip': '0xffffffffa46665ab', 'symbol': '_raw_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,118567342861740,795984,795988,3,extract,"[{'ip': '0xffffffffa45cec0b', 'symbol': 'clear...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,118567343521333,795984,795986,1,extract,"[{'ip': '0xffffffffa45cec0b', 'symbol': 'clear...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,118567347164207,795984,795984,0,extract,"[{'ip': '0xffffffffa4801280', 'symbol': 'asm_e...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29460,118604748562866,795984,795988,3,extract,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
29461,118604748874827,795984,795984,0,extract,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
29465,118604754501606,795984,795988,3,extract,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
29466,118604757599468,795984,795987,2,extract,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
